In [0]:
import pandas as pd
import numpy as np

In [0]:
from scipy import ndimage
from skimage.transform import resize
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from google.colab import files

In [0]:
from sklearn.metrics import confusion_matrix

In [5]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

Using TensorFlow backend.


In [0]:
URL_ENDPOINT = "http://cs.mcgill.ca/~ksinha4/datasets/kaggle/"

train_x = np.loadtxt(URL_ENDPOINT+"train_x.csv", delimiter=",")
train_y = np.loadtxt(URL_ENDPOINT+"train_y.csv", delimiter=",")
test_x = np.loadtxt(URL_ENDPOINT+"test_x.csv", delimiter=",")

In [0]:
x = train_x.reshape(-1, 64, 64) # reshape 
y = train_y.reshape(-1, 1) 
test = test_x.reshape(-1, 64, 64)

In [0]:
def find_largest_bounding_box(im):
  # convert to binary image
#   _, im = cv2.threshold(im,254,255,cv2.THRESH_BINARY)
  im[im<255]=0
  im[im==255]=1
  
  label_im, nb_labels = ndimage.label(im)
  
  # find largest connected component
  sizes = ndimage.sum(im, label_im, range(nb_labels + 1))
  mask_size = sizes < 4**2
  remove_pixel = mask_size[label_im]
  label_im[remove_pixel] = 0
  labels = np.unique(label_im)
  label_im = np.searchsorted(labels, label_im)

  digits = ndimage.find_objects(label_im)
  max_side = 0
  max_ind = 0
  for i in range(0,len(digits)):
    bound = im[digits[i]]
    if np.max(bound.shape) > max_side:
      max_side = np.max(bound.shape)
      max_ind = i

  bound_rec = im[digits[max_ind]]
  
  # pad and resize
  if bound_rec.shape[0] > bound_rec.shape[1]:
    resize_short = round(24*bound_rec.shape[1]/bound_rec.shape[0])
    bound_rec = resize(bound_rec, (24, resize_short), mode='reflect')
    pad_before = (np.ceil((28 - resize_short)/2)).astype(int)
    pad_after = (np.floor((28 - resize_short)/2)).astype(int)
    bound_squ = np.pad(bound_rec,((2,2),(pad_before, pad_after)),'constant', constant_values=0)
    
  else:
    resize_short = round(24*bound_rec.shape[0]/bound_rec.shape[1])
    bound_rec = resize(bound_rec, (resize_short, 24), mode='reflect')
    pad_before = (np.ceil((28 - resize_short)/2)).astype(int)
    pad_after = (np.floor((28 - resize_short)/2)).astype(int)
    bound_squ = np.pad(bound_rec,((pad_before, pad_after),(2,2)),'constant', constant_values=0)
#   _, bound_squ = cv2.threshold(bound_squ,254,255,cv2.THRESH_BINARY)
  bound_squ[bound_squ<1] = 0
  
  return bound_squ

In [0]:
def preprocess(train):
  prep = np.zeros((len(train),28,28))
  for i in range(0,len(prep)):
    prep[i] = find_largest_bounding_box(train[i].copy())
  return prep

In [11]:
prep_x = preprocess(x)
np.savetxt('prep_train_x.csv',prep_x.reshape(-1,28**2),delimiter=',')
prep_test = preprocess(test)
np.savetxt('prep_test_x.csv',prep_test.reshape(-1,28**2),delimiter=',')

/usr/local/lib/python3.6/dist-packages/scipy/ndimage/measurements.py:431: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  safe = ((np.issubdtype(dt, int) and dt.itemsize <= int_size) or


In [0]:
#retransform data input to prepare for 
prep_x = prep_x.reshape(prep_x.shape[0], 1, 28, 28).astype('float32')
prep_test = prep_test.reshape(prep_test.shape[0], 1, 28, 28).astype('float32')

In [0]:
train = prep_x[:40000]
#train = train.reshape(-1,28**2)
train_y = y[:40000]
valid = prep_x[40000:]
#valid = valid.reshape(-1,28**2)
valid_y = y[40000:]

train = prep_x[:40000]
#train = train.reshape(-1,28**2)
train_y = y[:40000]
valid = prep_x[40000:]
#valid = valid.reshape(-1,28**2)
valid_y = y[40000:]

In [0]:
y_train = np_utils.to_categorical(train_y)
y_test = np_utils.to_categorical(valid_y)
num_classes = y_test.shape[1]
real_y = np_utils.to_categorical(y)
num_classes = real_y.shape[1]

In [0]:
def baseline_model():
	# create model
	model = Sequential()
	model.add(Conv2D(28, (6, 6), input_shape=(1, 28, 28), activation='relu'))
	model.add(MaxPooling2D(pool_size=(3, 3)))
	model.add(Dropout(0.2))
	model.add(Flatten())
	model.add(Dense(128, activation='relu'))
	model.add(Dense(num_classes, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [0]:
def larger_model():
	# create model
	model = Sequential()
	model.add(Conv2D(30, (5, 5), input_shape=(1, 28, 28), activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Conv2D(15, (3, 3), activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.2))
	model.add(Flatten())
	model.add(Dense(128, activation='relu'))
# 	model.add(Dense(50, activation='relu'))
	model.add(Dense(num_classes, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [36]:
# build the model
model = baseline_model()
# Fit the model
model.fit(train, y_train, validation_data=(valid, y_test), epochs=10, batch_size=200, verbose=1)
# Final evaluation of the model
scores = model.evaluate(valid, y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 2s 62us/step - loss: 0.8425 - acc: 0.7491 - val_loss: 0.4544 - val_acc: 0.8783
Epoch 2/10
40000/40000 [==============================] - 2s 58us/step - loss: 0.4408 - acc: 0.8820 - val_loss: 0.3614 - val_acc: 0.9058
Epoch 3/10
40000/40000 [==============================] - 2s 58us/step - loss: 0.3673 - acc: 0.9043 - val_loss: 0.3091 - val_acc: 0.9229
Epoch 4/10
40000/40000 [==============================] - 2s 58us/step - loss: 0.3283 - acc: 0.9146 - val_loss: 0.2953 - val_acc: 0.9260
Epoch 5/10
40000/40000 [==============================] - 2s 58us/step - loss: 0.3034 - acc: 0.9213 - val_loss: 0.2764 - val_acc: 0.9311
Epoch 6/10
16600/40000 [===========>..................] - ETA: 1s - loss: 0.2758 - acc: 0.9294

40000/40000 [==============================] - 2s 58us/step - loss: 0.2803 - acc: 0.9296 - val_loss: 0.2680 - val_acc: 0.9350
Epoch 7/10
40000/40000 [==============================] - 2s 59us/step - loss: 0.2659 - acc: 0.9330 - val_loss: 0.2612 - val_acc: 0.9357
Epoch 8/10
40000/40000 [==============================] - 2s 58us/step - loss: 0.2524 - acc: 0.9362 - val_loss: 0.2594 - val_acc: 0.9361
Epoch 9/10
40000/40000 [==============================] - 2s 57us/step - loss: 0.2393 - acc: 0.9396 - val_loss: 0.2547 - val_acc: 0.9377
Epoch 10/10
40000/40000 [==============================] - 2s 58us/step - loss: 0.2270 - acc: 0.9432 - val_loss: 0.2559 - val_acc: 0.9355
CNN Error: 6.45%


In [70]:
# build the model
model = larger_model()
# Fit the model
model.fit(train, y_train, validation_data=(valid, y_test), epochs=10, batch_size=200, verbose=1)
# Final evaluation of the model
scores = model.evaluate(valid, y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 3s 81us/step - loss: 0.9160 - acc: 0.7156 - val_loss: 0.4524 - val_acc: 0.8788
Epoch 2/10
40000/40000 [==============================] - 3s 66us/step - loss: 0.4488 - acc: 0.8774 - val_loss: 0.3434 - val_acc: 0.9118
Epoch 3/10
40000/40000 [==============================] - 3s 66us/step - loss: 0.3680 - acc: 0.9035 - val_loss: 0.3015 - val_acc: 0.9247
Epoch 4/10
40000/40000 [==============================] - 3s 67us/step - loss: 0.3275 - acc: 0.9157 - val_loss: 0.2788 - val_acc: 0.9305
Epoch 5/10
40000/40000 [==============================] - 3s 66us/step - loss: 0.3044 - acc: 0.9228 - val_loss: 0.2671 - val_acc: 0.9349
Epoch 6/10
 6800/40000 [====>.........................] - ETA: 2s - loss: 0.2846 - acc: 0.9271

40000/40000 [==============================] - 3s 67us/step - loss: 0.2855 - acc: 0.9267 - val_loss: 0.2591 - val_acc: 0.9381
Epoch 7/10
40000/40000 [==============================] - 3s 67us/step - loss: 0.2712 - acc: 0.9308 - val_loss: 0.2535 - val_acc: 0.9362
Epoch 8/10
40000/40000 [==============================] - 3s 66us/step - loss: 0.2560 - acc: 0.9340 - val_loss: 0.2476 - val_acc: 0.9411
Epoch 9/10
40000/40000 [==============================] - 3s 68us/step - loss: 0.2459 - acc: 0.9374 - val_loss: 0.2433 - val_acc: 0.9412
Epoch 10/10
40000/40000 [==============================] - 3s 67us/step - loss: 0.2383 - acc: 0.9387 - val_loss: 0.2372 - val_acc: 0.9431
CNN Error: 5.69%


In [0]:
#SAVES THE PREDICTIONS FOR TEST SET

predictions = model.predict(prep_test)
predictions2 = model.predict_classes(prep_test)
predictions3 = predictions2.reshape(predictions2.shape[0], 1)
result = np.concatenate((np.asarray(range(0,len(predictions3))).reshape(len(predictions3),1),predictions3), axis=1).astype(int)
np.savetxt('test_y.csv',result,delimiter=',',header='Id,Label', fmt='%f')
files.download('test_y.csv')